Proof of Concept: Voice Assistant with Cognitive AI to help complete Warehouse Operations.

Business Requirement:  
1.  Get the input from User over microphone, to Text and interface with SAP EWM as input.
2.  Voice request for  inputs and promots for Error for user corrections.
3.  Train the AI model with L1/L2 Support, assisting the opertors 
    on realtime exception handling and workarounds.

Developer           :   ROKKAMS(s.rokkam@sap.com).
GitReposit         :   https://github.com/sriramrokkam/texttospeech

# Backend Technologies:  
   1. Azure -Cognitive AI      for NLP Services
   2. SAP S4 Embedded EWM 9.3   for Warehouse Operations.
   3. SAP ABAP                  for Server Side Validation.
    4. Python 3.3               for Client Side Validations
#Integration Technologies:
   1.Azure Event Hub
   2.SAP ABAP Webhooks
   3.Restful API's. 
                    
#Frontend Technologies:
   1. SAP AppGyver
   2. Django / CSS / JS

In [9]:
# Import namespaces
from dotenv import load_dotenv
from datetime import datetime
import os
import azure.cognitiveservices.speech as speech_sdk

In [10]:
# Get Configuration Settings
load_dotenv()
GV_COGKEY = os.getenv('COG_SERVICE_KEY')
GV_COGREG = os.getenv('COG_SERVICE_REGION')

In [11]:
#New Class for initiating Speech_Config
class get_speech:
    def __init__(self, GV_COGKEY, GV_COGREG):
    # Configure speech service
        global speech_config
    speech_config = speech_sdk.SpeechConfig(GV_COGKEY, GV_COGREG)
    print('Ready to use speech service in:', speech_config.region)

obj_getsp = get_speech(GV_COGKEY, GV_COGREG)

Ready to use speech service in: eastasia


In [15]:
# Transcribe
class S2T:
    def TranscribeCommand(self):
       global command
    # automatic language detection.
    auto_detect_source_language_config = speech_sdk.languageconfig.AutoDetectSourceLanguageConfig()
    # Configure speech recognition
    audio_config = speech_sdk.AudioConfig(use_default_microphone=True)
    speech_recognizer = speech_sdk.SpeechRecognizer(obj_getsp.speech_config, audio_config)
    print('Speak now...')

    # Process speech input
    speech = speech_recognizer.recognize_once_async().get()
    if speech.reason == speech_sdk.ResultReason.RecognizedSpeech:
        command = speech.text
        print(command)
    else:
        print(speech.reason)
    if speech.reason == speech_sdk.ResultReason.Canceled:
        cancellation = speech.cancellation_details
        print(cancellation.reason)
        print(cancellation.error_details)     

obj_s2t = S2T()
obj_s2t.TranscribeCommand()

Speak now...
Hello. Hello. Hello.


In [28]:
# New Text to Speech Class.
class T2S:
    def detect_language(self,text):
         # create the auto detection language configuration without specific languages
        source_language = speech_sdk.languageconfig.AutoDetectSourceLanguageConfig()
        print (source_language)

    # Creates a speech synthesizer using the default speaker as audio output.
        speech_synthesizer = speech_sdk.SpeechSynthesizer( speech_config=obj_getsp.speech_config, auto_detect_source_language_config=source_language)
        result = speech_synthesizer.speak_text_async(obj_s2t.command).get()
        print(result)

    # Check result
        if result.reason == speech_sdk.ResultReason.SynthesizingAudioCompleted:
            print("Speech synthesized to speaker for text [{}]".format(text))
        stream = speech_sdk.AudioDataStream(result)
        stream.save_to_wav_file(r"C:\output.wav")


'''read_text = input("Enter Text to Read :")'''
read_text = 'Hello World !'
#read_text = obj_s2t.command
obj_t2s = T2S()
obj_t2s.detect_language(read_text)     

SpeechSynthesisResult(result_id=29ad235c84c549a48da08acfcc69db53, reason=ResultReason.SynthesizingAudioCompleted, audio_length=139246)
Speech synthesized to speaker for text [Hello World !]


RuntimeError: Exception with error code: 
[CALL STACK BEGIN]

    > pal_string_to_wstring
    - pal_string_to_wstring
    - audio_data_stream_save_to_wave_file
    - ffi_prep_go_closure
    - ffi_call_go
    - ffi_call
    - 00007FFA8BD82C35 (SymFromAddr() error: Attempt to access invalid address.)
    - 00007FFA8BD829B7 (SymFromAddr() error: Attempt to access invalid address.)
    - 00007FFA8BD82628 (SymFromAddr() error: Attempt to access invalid address.)
    - PyObject_Call
    - PyEval_EvalFrameDefault
    - PyFunction_Vectorcall
    - PyVectorcall_Call
    - PyObject_Call
    - PyEval_EvalFrameDefault
    - PyFunction_Vectorcall

[CALL STACK END]

Exception with an error code: 0x8 (SPXERR_FILE_OPEN_FAILED)